In [1]:
import pandas as pd

In [2]:
RANK = ['A','B','C','D']
NONE_RANK = 'N'
excel_file = pd.read_excel("./couplet/RNN训练集.xlsx",sheet_name=None)

In [30]:
# Vectorize the data.
voc_dict = []
percentage_list = []
samples = []
per_percnetage = 1/len(RANK)
for i in range(len(RANK)):
    percentage_list.append(per_percnetage*(i+1))

for sheet_name,sheet_content in excel_file.items():
    point_list = []
    for i in range(len(percentage_list)):
        point = sheet_content.loc[:,['评分']].quantile(percentage_list[i])
        point_list.append(float(point))
    
    sample_dict = {}
    for row in sheet_content.loc[:,['商铺顺序号','商铺类型','评分']].values:
        if pd.isnull(row[2]):
            continue
        street_name = row[0][0]
        if street_name not in sample_dict:
            sample_dict[street_name] = ["",""]
        if pd.isnull(row[2]):
            sample_dict[street_name][0] += row[1]
            sample_dict[street_name][0] += " "
            sample_dict[street_name][1] += NONE_RANK
            sample_dict[street_name][1] += " "
        else:
            for i in range(len(point_list)):
                if float(row[2]) <= point_list[i]:
                    sample_dict[street_name][0] += row[1]
                    sample_dict[street_name][0] += " "
                    sample_dict[street_name][1] += RANK[i]
                    sample_dict[street_name][1] += " "
                    break
    for sample in sample_dict.values():
        sample[0] = sample[0][:-1].lower()
        sample[1] = sample[1][:-1].lower()
        samples.append(sample)
        sample_inv = ["",""]
        for i in sample[0].split(" "):
            voc_dict.append(i.lower())
            sample_inv[0] = i + " " + sample_inv[0]
        for i in sample[1].split(" "):
            voc_dict.append(i.lower())
            sample_inv[1] = i + " " + sample_inv[1]
        sample_inv[0] = sample_inv[0][:-1].lower()
        sample_inv[1] = sample_inv[1][:-1].lower()
        samples.append(sample_inv)

voc_dict = set(voc_dict)

In [31]:
print(samples[0])
print(samples[1])

['ffr cfr cfr js p dh fls cs f ch sts ffr cfr ffr dh sts cfr cs cs cfr', 'a c c b a d a a d d a d b d c c c a a c']
['cfr cs cs cfr sts dh ffr cfr ffr sts ch f cs fls dh p js cfr cfr ffr', 'c a a c c c d b d a d d a a d a b c c a']


In [19]:
print(voc_dict)

{'dh', 'h', 'f', 'ffr', 'as', 'js', 'eg', 'fls', 'con', 'd', 'sp', 'fs', 'ba', 'fis', 'sts', 'ps', 'bo', 'c', 'pcis', 'sh', 'vh', 'sa', 'cfr', 'a', 'cs', 'os', 'p', 'is', 'sps', 'tc', 'cos', 'ss', 'b', 'heh', 'hc', 'sm', 'ch', 'th'}


In [36]:
with open(f"./couplet/train/pos.txt",encoding="utf-8",mode ="w") as posfile,\
    open(f"./couplet/train/rank.txt",encoding="utf-8",mode ="w") as rankfile:
    for sample in samples:
        posfile.write(sample[0] + "\n")
        rankfile.write(sample[1]+ "\n")

with open(f"./couplet/voc",encoding="utf-8",mode ="w") as vocfile:
    vocfile.write("<s>\n</s>\n,\n。\n")
    for word in voc_dict:
        vocfile.write(word + "\n")

In [6]:
test = "1 2 3 4 5"
tl = []
for i in test.split(" "):
    tl.insert(0,i)
print(tl)

['5', '4', '3', '2', '1']


In [ ]:
SPLIT_CHARS = ['，', '、', ',', '.', '。', '!', '！', '?', '？', ' ']


def all_same(s):
    if len(s) <= 1:
        return True
    for i in range(1, len(s)):
        if s[i] not in SPLIT_CHARS and s[i] != s[0]:
            return False
    return True

def sort_outputs(outputs, scores):
    new_scores, new_outputs = zip(*sorted(zip(scores, outputs), reverse=True))
    return list(new_outputs), list(new_scores)

def manual_correct_result(in_str, outputs, scores):
    is_all_same = all_same(in_str)
    for i in range(len(outputs)):
        if is_all_same:
            scores[i] -= 100
            continue
        # print(abs(len(in_str.split(" ")) - len(outputs[i])))
        scores[i] -= abs(len(in_str.split(" ")) - len(outputs[i]))*1000
        
    return outputs, scores

def infer(in_str):
    if len(in_str) == 0 or len(in_str) > 50:
        return [u'您的输入太长了'], []
    else:
        model_outputs, model_scores = m.infer(in_str)
        # print(model_outputs,model_scores)
        # print(model_outputs)
        model_scores = model_scores.tolist()
        unsorted_outputs, unsorted_scores = manual_correct_result(in_str, model_outputs, model_scores)
        output, score = sort_outputs(unsorted_outputs, unsorted_scores)
        best_output = ""
        for rank in output[0]:
            best_output = best_output + rank +" "
        best_output = best_output[:-1]

        print('输入：%s；输出：%s ; score: %s' % (
            in_str, best_output, score[0]))

In [99]:
from model import Model

m = Model(
        './couplet/train/pos.txt',##训练输入
        './couplet/train/rank.txt',##训练输出
        './couplet/train/pos.txt',##测试输入
        './couplet/train/rank.txt',##测试输出，应该是空
        './couplet/v1',
        num_units=1024, layers=4, dropout=0.2,
        batch_size=32, learning_rate=0.001,
        output_dir='./output_couplet',
        restore_model=False,save_step = 5,init_infer = True)

m.train(600)

INFO:tensorflow:Restoring parameters from ./output_couplet\model.ckpl


KeyboardInterrupt: 

In [98]:
infer("cos cs ch ba sa ch ch sa cos")
# c a c a c d c d a c c d a a

0
0
0
0
0
0
0
0
0
0
输入：cos cs ch ba sa ch ch sa cos；输出：a d b d c a d c a ; score: -0.4640083312988281
